<a href="https://colab.research.google.com/github/randiepathirage/4th/blob/main/imp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import re
import matplotlib.pyplot as plt
from tensorflow import keras
from unicodedata import normalize
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.layers import LSTM,Bidirectional,Input,Concatenate
from tensorflow.keras.optimizers import Adam

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# DATA_PATH = '/content/drive/MyDrive/UNI/4th Year/Research/DataSet/LTRL_Clean.tsv'
#DATA_PATH = '/content/drive/MyDrive/4th year Research/data collection/data/LTRL_Parallel_Words_3_Shuffled.csv'
DATA_PATH = '/content/LTRL_Parallel_Words_3_Shuffled.csv'


model_id = "e5_1"
batch_size = 32  # Batch size for training.
epochs = 50 # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 24415  # Number of samples to train on.
test_count = 5000

df = pd.read_csv(DATA_PATH, names=["singlish","sinhala"],dtype=str).astype(str)
df = df.sample(frac=1).reset_index(drop=True)

input_data = pd.DataFrame(df.singlish.values[:num_samples])
input_data.columns = ['singlish']

target_data = pd.DataFrame(df.sinhala.values[:num_samples])
target_data.columns = ['sinhala']

#@title Preview dataset
preview_dataset = True #@param {type:"boolean"}

if(preview_dataset):
  print("Singlish")
  print(input_data.head())
  print("Sinhala")
  print(target_data.head())

FileNotFoundError: ignored

In [ ]:
def normalize_text(text):
  text = normalize('NFD', text).encode('utf-8')
  return text.decode('UTF-8')

def clean_and_normalize(text):
  text = normalize_text(text.lower())
  return text

def print_sentence_length_status(seqeunce):
  lt10 = lt20 = lt30 = lt40 = lt50 = lg50 = 0
  for line in seqeunce:
    c = len(line.split())
    if(c<=10):
      lt10+=1
    elif(c<=20):
      lt20+=1
    elif(c<=30):
      lt30+=1
    elif(c<=40):
      lt40+=1
    elif(c<=50):
      lt50+=1
    else:
      lg50+=1
 
  print("LOE10 : ",lt10)
  print("LOE20 : ",lt20)
  print("LOE30 : ",lt30)
  print("LOE40 : ",lt40)
  print("LOE50 : ",lt50)
  print("GTE50  : ",lg50)
  print("Total sentence count : ", lt10+lt20+lt30+lt40+lt50+lg50)

In [ ]:
#@title Default title text
print_stat = True #@param {type:"boolean"}
if(print_stat):
  print_sentence_length_status(input_data['singlish'])

In [ ]:
def removeUnnecessaryRepeats(word):
  prev = word[0]
  repeatCount = 1
  ans = word[0]
  for i in range(1,len(word)):
    if(word[i]==prev):
      repeatCount+=1
    else:
      repeatCount=1
    
    if(repeatCount<3):
      ans+=word[i]
      prev = word[i]

  return ans

def preProcessSinglish(singText):
  vowels =  ['a','e','i','o','u']
  specialConsonentSuffix = ['h']
  singText = singText.lower()
  singWords = singText.split()
  finList = []
  for singWord in singWords:
    singWord = removeUnnecessaryRepeats(singWord)
    buf = ""
    for i in range(len(singWord)):
      char = singWord[i]

      if(char in vowels):
        if(i<len(singWord)-1):
          if(char == singWord[i+1]):
            buf+=char
          else:
            buf+=char
            finList.append(buf)
            buf=""
        else:
          buf+=char
          finList.append(buf)
          buf=""
      else:
        if(i<len(singWord)-1):
          if(char == singWord[i+1] or (singWord[i+1] not in vowels and singWord[i+1] not in specialConsonentSuffix)):
            buf+=char
            finList.append(buf)
            buf=""
          else:
            buf+=char
        else:
          buf+=char
          finList.append(buf)
          buf=""
    finList.append("$")

  return " ".join(finList)

def preProcessSinhala(sinText):
  sinWords = sinText.split()
  finList = []
  for sinWord in sinWords:
    charList = re.findall(r'\w\W?',sinWord.strip())
    finList.extend(charList)
    finList.append("$")
  return " ".join(finList)

input_data['singlish'] = input_data['singlish'].apply(lambda x: preProcessSinglish(clean_and_normalize(x)))
target_data['sinhala'] = target_data['sinhala'].apply(lambda x: preProcessSinhala(clean_and_normalize(x))).apply(lambda x: '<S> '+x+' <E>')


In [ ]:
#@title View Dataset Heads
view_dataset_head = True #@param {type:"boolean"}
if(view_dataset_head):
  print(input_data['singlish'].head)

In [ ]:
max_encoder_length = 250
max_decoder_length = 250

# Vectorize the data.
input_texts = []
target_texts = []
input_TUs = set()
target_TUs = set()

input_texts = input_data.singlish
target_texts = target_data.sinhala

for input_text in input_texts:
  for TU in input_text.split():
      if TU not in input_TUs:
          input_TUs.add(TU)
  
for target_text in target_texts:
  for TU in target_text.split():
      if TU not in target_TUs:
          target_TUs.add(TU)

input_TUs = sorted(list(input_TUs))
target_TUs = sorted(list(target_TUs))
num_encoder_tokens = len(input_TUs)
num_decoder_tokens = len(target_TUs)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(TU, i) for i, TU in enumerate(input_TUs)])
target_token_index = dict([(TU, i) for i, TU in enumerate(target_TUs)])

In [ ]:
input_texts

In [ ]:
target_token_index

In [ ]:
input_token_index

In [ ]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, TU in enumerate(input_text.split()):
        encoder_input_data[i, t, input_token_index[TU]] = 1.0
    for t, TU in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[TU]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[TU]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index["$"]] = 1.0
    decoder_target_data[i, t:, target_token_index["$"]] = 1.0

target_texts[0].split()

encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.Bidirectional(keras.layers.LSTM(latent_dim, return_state=True))
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder(encoder_inputs)
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
encoder_states = [state_h, state_c]

print(state_h.shape)

decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

decoder_lstm = keras.layers.LSTM(latent_dim*2, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

training_limit = num_samples-test_count
# Adam(learning_rate=3e-4)
model.compile(
    optimizer=Adam(learning_rate=3e-4), loss="categorical_crossentropy", metrics=["categorical_accuracy"], 
)

In [ ]:
model.summary()

In [ ]:
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [ ]:
reverse_target_char_index

In [ ]:
target_token_index

In [ ]:
history = model.fit(
    [encoder_input_data[:training_limit], decoder_input_data[:training_limit]],
    decoder_target_data[:training_limit],
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
    callbacks=[model_checkpoint_callback]
)

plt.plot(history.history['loss'], label='Loss (training data)')
plt.plot(history.history['val_loss'], label='Loss (validation data)')
plt.title('Singlish to Sinhala Transliteration')
plt.ylabel('Loss value')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.savefig('model_plot.png')

model.save(model_id)

target_token_index

In [ ]:
model = keras.models.load_model(model_id)

encoder_inputs = model.input[0]  # input_1
encoder_outputs, forward_h, forward_c, backward_h, backward_c = model.layers[1].output  # lstm_1
state_h_enc = Concatenate()([forward_h, backward_h])
state_c_enc = Concatenate()([forward_c, backward_c])
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim*2,))
decoder_state_input_c = keras.Input(shape=(latent_dim*2,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[5]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[6]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)


    target_seq = np.zeros((1, 1, num_decoder_tokens))
    
    target_seq[0, 0, target_token_index["<S>"]] = 1.0


    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]

        if sampled_char == "<E>" or len(decoded_sentence) > max_decoder_seq_length:
          stop_condition = True
        else:
          decoded_sentence += sampled_char
          decoded_sentence += " "

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        states_value = [h, c]
    return decoded_sentence

reverse_index_df = pd.DataFrame(reverse_target_char_index.items())
reverse_index_df.to_csv (r'reverse_index_df.csv', index = False, header=False)

input_token_index_df = pd.DataFrame.from_dict(input_token_index.items())
input_token_index_df.to_csv (r'input_token_index.csv', index = False, header=False)

target_token_index_df = pd.DataFrame.from_dict(target_token_index.items())
target_token_index_df.to_csv (r'target_token_index.csv', index = False, header=False)

unseen_file = open("unseen_data.txt", "w")
unseen_data_target = []
unseen_data_candidate = []
i = 0
for seq_index in range(num_samples-test_count,num_samples):
    i+=1
    #print("unseen data ",i)
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    # print(input_seq.shape)
    decoded_sentence = decode_sequence(input_seq)
    unseen_file.write("-\n")
    input_sentence = input_texts[seq_index]
    target_sentence = target_texts[seq_index]
    target_sentence = target_sentence.replace("<S>", "")
    target_sentence = target_sentence.replace("<E>", "")

    unseen_data_target.append(target_sentence)
    unseen_data_candidate.append(decoded_sentence)

    input_sentence = input_sentence.replace(" ","")
    input_sentence = input_sentence.replace("$"," ")

    unseen_file.write(input_sentence+"\n")
    unseen_file.write(target_sentence + "\n")
    unseen_file.write(decoded_sentence+"\n")

df_unseen_target = pd.DataFrame(unseen_data_target)
df_unseen_candidate = pd.DataFrame(unseen_data_candidate)
df_unseen_target.to_csv (r'unseen_target.csv', index = False, header=False)
df_unseen_candidate.to_csv (r'unseen_candidate.csv', index = False, header=False)
unseen_file.close()

seen_file = open("seen_data.txt", "w")
seen_data_target = []
seen_data_candidate = []
i=0
for seq_index in range(min(2000,num_samples - test_count)):
    i+=1
    print("seen data ",i)
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    # print(input_seq)
    decoded_sentence = decode_sequence(input_seq)
    input_sentence = input_texts[seq_index]
    target_sentence = target_texts[seq_index]
    target_sentence = target_sentence.replace("<S>", "")
    target_sentence = target_sentence.replace("<E>", "")

    seen_data_target.append(target_sentence)
    seen_data_candidate.append(decoded_sentence)

    input_sentence = input_sentence.replace(" ","")
    input_sentence = input_sentence.replace("$"," ")

    seen_file.write("-\n")
    seen_file.write(input_sentence + "\n")
    seen_file.write(target_sentence + "\n")
    seen_file.write(decoded_sentence + "\n")

df_seen_target = pd.DataFrame(seen_data_target)
df_seen_candidate = pd.DataFrame(seen_data_candidate)
df_seen_target.to_csv (r'seen_target.csv', index = False, header=False)
df_seen_candidate.to_csv (r'seen_candidate.csv', index = False, header=False)
seen_file.close()

def process_text(text):
  # breaak into words
  words = text.split()
  for word in words:
    # convert to TUs
    preprocessed_word = preProcessSinglish(clean_and_normalize(word))
    # create one hot embedding
    encoded_input = np.zeros(
        (1, max_encoder_seq_length, num_encoder_tokens), dtype="float32"
    )
    for t, TU in enumerate(input_text.split()):
        encoded_input[0, t, input_token_index[TU]] = 1.0
    encoded_input[0, t + 1 :, input_token_index["$"]] = 1.0
    # predict
    decoded_sentence = decode_sequence(encoded_input)

In [ ]:
from nltk.translate.bleu_score import corpus_bleu
score = corpus_bleu(unseen_data_target, unseen_data_candidate)
print(score)

In [ ]:
def removeUnnecessaryRepeats(word):
  prev = word[0]
  repeatCount = 1
  ans = word[0]
  for i in range(1,len(word)):
    if(word[i]==prev):
      repeatCount+=1
    else:
      repeatCount=1
    
    if(repeatCount<3):
      ans+=word[i]
      prev = word[i]

  return ans

In [ ]:
def preProcessSinglish(singText):
  vowels =  ['a','e','i','o','u']
  specialConsonentSuffix = ['h']
  singText = singText.lower()
  singWords = singText.split()
  finList = []
  for singWord in singWords:
    singWord = removeUnnecessaryRepeats(singWord)
    buf = ""
    for i in range(len(singWord)):
      char = singWord[i]

      if(char in vowels):
        if(i<len(singWord)-1):
          if(char == singWord[i+1]):
            buf+=char
          else:
            buf+=char
            finList.append(buf)
            buf=""
        else:
          buf+=char
          finList.append(buf)
          buf=""
      else:
        if(i<len(singWord)-1):
          if(char == singWord[i+1] or (singWord[i+1] not in vowels and singWord[i+1] not in specialConsonentSuffix)):
            buf+=char
            finList.append(buf)
            buf=""
          else:
            buf+=char
        else:
          buf+=char
          finList.append(buf)
          buf=""
    finList.append("$")

  return " ".join(finList)

In [ ]:
preProcessSinglish("aakaaarura")